# IMPORT DES LIBRAIRIES 

In [486]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

# OUVERTURE DU JEU DE DONNEES

In [487]:
df = pd.read_csv('../DATA/OUT/dataset_with_data_prepocessing.csv')
df.head()

C:\Users\jonat\AppData\Local\Temp\ipykernel_11460\3720063734.py:1: DtypeWarning:

Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.



,NoDisp,DateMutation,ValeurFonciere,NoVoie,BTQ,TypeVoie,CodeVoie,Voie,CP,Commune,...,NatureCultureSpeciale,SurfaceTerrain,slice,quarter,Year,Month,AvecTerrain,SurfaceBatiCat,SurfaceCarrez,Localisation
0,1,2021-04-26,311650.0,16,NaN,RUE,0270,DE LA GRANGE DIMIERE,33850,LEOGNAN,...,NaN,0.0,"[300000.0, 400000.0)",2,2021,4,False,"(80.0, 90.0]",84.07,33238000AA
1,1,2021-04-26,122000.0,28,NaN,CRS,0840,DE VERDUN,33470,GUJAN MESTRAS,...,NaN,0.0,"[100000.0, 150000.0)",2,2021,4,False,"(0.999, 35.0]",0.00,33199000BO
2,1,2021-04-23,420612.0,47,NaN,RUE,0144,JEAN MARIE PELT,33380,MIOS,...,NaN,607.0,"[400000.0, 500000.0)",2,2021,4,True,"(118.0, 144.0]",0.00,33284000CT
3,1,2021-04-19,167000.0,1,NaN,RUE,0030,ARMAND DANEY,33470,GUJAN MESTRAS,...,NaN,0.0,"[150000.0, 200000.0)",2,2021,4,False,"(47.0, 60.0]",49.14,33199000BO
4,1,2021-05-05,450000.0,11,NaN,ALL,1180,FRERES DUPUY CAZAUX,33260,LA TESTE-DE-BUCH,...,NaN,684.0,"[400000.0, 500000.0)",2,2021,5,True,"(100.0, 118.0]",0.00,33529000CT


# CREATION D'UN ID PAR INDIVIDU

In [488]:
# Ajout d'une colonne ID qui sera un identifiant unique pour chaque ligne (commençant à 1)
df['ID'] = range(1, len(df) + 1)

# Vérification de l'ajout de la colonne ID
df[['ID'] + df.columns.tolist()[:-1]].head()

,ID,NoDisp,DateMutation,ValeurFonciere,NoVoie,BTQ,TypeVoie,CodeVoie,Voie,CP,...,NatureCultureSpeciale,SurfaceTerrain,slice,quarter,Year,Month,AvecTerrain,SurfaceBatiCat,SurfaceCarrez,Localisation
0,1,1,2021-04-26,311650.0,16,NaN,RUE,0270,DE LA GRANGE DIMIERE,33850,...,NaN,0.0,"[300000.0, 400000.0)",2,2021,4,False,"(80.0, 90.0]",84.07,33238000AA
1,2,1,2021-04-26,122000.0,28,NaN,CRS,0840,DE VERDUN,33470,...,NaN,0.0,"[100000.0, 150000.0)",2,2021,4,False,"(0.999, 35.0]",0.00,33199000BO
2,3,1,2021-04-23,420612.0,47,NaN,RUE,0144,JEAN MARIE PELT,33380,...,NaN,607.0,"[400000.0, 500000.0)",2,2021,4,True,"(118.0, 144.0]",0.00,33284000CT
3,4,1,2021-04-19,167000.0,1,NaN,RUE,0030,ARMAND DANEY,33470,...,NaN,0.0,"[150000.0, 200000.0)",2,2021,4,False,"(47.0, 60.0]",49.14,33199000BO
4,5,1,2021-05-05,450000.0,11,NaN,ALL,1180,FRERES DUPUY CAZAUX,33260,...,NaN,684.0,"[400000.0, 500000.0)",2,2021,5,True,"(100.0, 118.0]",0.00,33529000CT


# DEFINITION DES OUTLIERS POUR LES SURFACES DE TERRAINS PAR RAPPORT A LEUR PRIX

### Calcul D’Outliers :
X < q1 - 1.5 x iqr X > q3 + 1.5 x iqr

# CALCUL DU PRIX TERRAIN AU M²

In [489]:
# ISOLER LES BIENS AVEC TERRAINS
bien_avec_terrain = df.loc[(df['SurfaceTerrain'] > 0)]

df['PrixTerrainM2'] = df['ValeurFonciere'] / df['SurfaceTerrain']
bien_avec_terrain['PrixTerrainM2'] = bien_avec_terrain['ValeurFonciere'] / bien_avec_terrain['SurfaceTerrain']
# Remplacez les valeurs infinies par zéro.
df['PrixTerrainM2'].replace(np.inf, 0, inplace=True)
bien_avec_terrain['PrixTerrainM2'].replace(np.inf, 0, inplace=True)

C:\Users\jonat\AppData\Local\Temp\ipykernel_11460\1641062933.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jonat\AppData\Local\Temp\ipykernel_11460\1641062933.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [490]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_terrain = bien_avec_terrain['PrixTerrainM2'].quantile([.25])
q1_prix_terrain = q1_prix_terrain[0.25]
q3_prix_terrain = bien_avec_terrain['PrixTerrainM2'].quantile([.75])
q3_prix_terrain = q3_prix_terrain[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_terrain = q3_prix_terrain - q1_prix_terrain

print(f"q1 = {q1_prix_terrain}, q3 = {q3_prix_terrain}, IQR = {iqr_prix_terrain}")

q1 = 299.6560426709942, q3 = 1302.0833333333333, IQR = 1002.427290662339


# CALCUL DES VALEURS LIMITE OUTLIERS

In [491]:
outliers_prix_terrain_m2_inferieur = q1_prix_terrain - (1.5 * iqr_prix_terrain)
outliers_prix_terrain_m2_superieur = q3_prix_terrain + (1.5 * iqr_prix_terrain)

print(f'Outliers Inférieur = {outliers_prix_terrain_m2_inferieur}')
print(f'Outliers Supérieur = {outliers_prix_terrain_m2_superieur}')

Outliers Inférieur = -1203.9848933225144
Outliers Supérieur = 2805.7242693268417


# COMPATAGE DES OUTLIERS

In [492]:
nb_outliers_surface_terrain = df.loc[(df['PrixTerrainM2'] > outliers_prix_terrain_m2_superieur) | (df['PrixTerrainM2'] < outliers_prix_terrain_m2_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_surface_terrain[0]} outliers.')

Nous aurions donc 13041 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [493]:
outliers_surface_terrain = df.loc[(df['PrixTerrainM2'] > outliers_prix_terrain_m2_superieur) | (df['PrixTerrainM2'] < outliers_prix_terrain_m2_inferieur)]

# DEFINITION DES OUTLIERS POUR LES SURFACES BATI PAR RAPPORT A LEUR PRIX
# CALCUL DU PRIX BATI AU M²

In [494]:
df['PrixBatiM2'] = df['ValeurFonciere'] / df['SurfaceReelleBati']

# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [495]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_bati = df['PrixBatiM2'].quantile([.25])
q1_prix_bati = q1_prix_bati[0.25]
q3_prix_bati = df['PrixBatiM2'].quantile([.75])
q3_prix_bati = q3_prix_bati[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_bati = q3_prix_bati - q1_prix_bati

print(f"q1 = {q1_prix_bati}, q3 = {q3_prix_bati}, IQR = {iqr_prix_bati}")

q1 = 2335.294117647059, q3 = 5000.0, IQR = 2664.705882352941


# CALCUL DES VALEURS LIMITE OUTLIERS

In [496]:
outliers_prix_bati_m2_inferieur = q1_prix_bati - (1.5 * iqr_prix_bati)
outliers_prix_bati_m2_superieur = q3_prix_bati + (1.5 * iqr_prix_bati)

print(f'Outliers Inférieur = {outliers_prix_bati_m2_inferieur}')
print(f'Outliers Supérieur = {outliers_prix_bati_m2_superieur}')

Outliers Inférieur = -1661.7647058823527
Outliers Supérieur = 8997.058823529413


# COMPATAGE DES OUTLIERS

In [497]:
nb_outliers_surface_bati = df.loc[(df['PrixBatiM2'] > outliers_prix_bati_m2_superieur) | (df['PrixBatiM2'] < outliers_prix_bati_m2_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_surface_bati[0]} outliers.')

Nous aurions donc 17988 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [498]:
outliers_surface_bati = df.loc[(df['PrixBatiM2'] > outliers_prix_bati_m2_superieur) | (df['PrixBatiM2'] < outliers_prix_bati_m2_inferieur)]

# DEFINITION DES OUTLIERS POUR LES SURFACES CARREZ PAR RAPPORT A LEUR PRIX
# CALCUL DU PRIX CARREZ AU M²

In [499]:
df['SurfaceCarrez'] = df['SurfaceCarrez1erLot'] + df['SurfaceCarrez2emeLot'] + df['SurfaceCarrez3emeLot'] + df['SurfaceCarrez4emeLot'] + df['SurfaceCarrez5emeLot']

In [500]:
# ISOLER LES BIENS AVEC UNE SURFACE CARREZ
bien_avec_carrez = df.loc[(df['SurfaceCarrez'] > 0)]

df['PrixCarrezM2'] = df['ValeurFonciere'] / df['SurfaceCarrez']
bien_avec_carrez['PrixCarrezM2'] = bien_avec_carrez['ValeurFonciere'] / bien_avec_carrez['SurfaceCarrez']
# Remplacez les valeurs infinies par zéro.
df['PrixCarrezM2'].replace(np.inf, 0, inplace=True)
bien_avec_carrez['PrixCarrezM2'].replace(np.inf, 0, inplace=True)

C:\Users\jonat\AppData\Local\Temp\ipykernel_11460\3913535154.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jonat\AppData\Local\Temp\ipykernel_11460\3913535154.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [501]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_prix_carrez = bien_avec_carrez['PrixCarrezM2'].quantile([.25])
q1_prix_carrez = q1_prix_carrez[0.25]
q3_prix_carrez = bien_avec_carrez['PrixCarrezM2'].quantile([.75])
q3_prix_carrez = q3_prix_carrez[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_prix_carrez = q3_prix_carrez - q1_prix_carrez

print(f"q1 = {q1_prix_carrez}, q3 = {q3_prix_carrez}, IQR = {iqr_prix_carrez}")

q1 = 2830.0179924446065, q3 = 4811.060153990553, IQR = 1981.0421615459463


# CALCUL DES VALEURS LIMITE OUTLIERS

In [502]:
outliers_prix_carrez_m2_inferieur = q1_prix_carrez - (1.5 * iqr_prix_carrez)
outliers_prix_carrez_m2_superieur = q3_prix_carrez + (1.5 * iqr_prix_carrez)

print(f'Outliers Inférieur = {outliers_prix_carrez_m2_inferieur}')
print(f'Outliers Supérieur = {outliers_prix_carrez_m2_superieur}')

Outliers Inférieur = -141.54524987431296
Outliers Supérieur = 7782.623396309473


# COMPATAGE DES OUTLIERS

In [503]:
nb_outliers_surface_carrez = df.loc[(df['PrixCarrezM2'] > outliers_prix_carrez_m2_superieur) | (df['PrixCarrezM2'] < outliers_prix_carrez_m2_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_surface_carrez[0]} outliers.')

Nous aurions donc 1379 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [504]:
outliers_surface_carrez = df.loc[(df['PrixCarrezM2'] > outliers_prix_carrez_m2_superieur) | (df['PrixCarrezM2'] < outliers_prix_carrez_m2_inferieur)]

# COMPATEGE DU TOTAL D'OUTLIERS DEPUIS LE DEBUT (DOUBLONS INCLUS)

In [505]:
nb_outliers_surface_terrain[0] + nb_outliers_surface_bati[0] + nb_outliers_surface_carrez[0]

32408

# CREATION D'UN DATAFRAME HORS OUTLIERS ET DOUBLONS D'OUTLIERS

In [506]:
# Concaténation des ID des outliers
ids_outliers = pd.concat([outliers_surface_bati['ID'], outliers_surface_terrain['ID'], outliers_surface_carrez['ID']])

# Suppression des duplicatas
ids_outliers = ids_outliers.drop_duplicates()

# Filtrage de df pour exclure les outliers
df_clean = df[~df['ID'].isin(ids_outliers)]

# CREATION D'UN CSV DEPUIS CE DATAFRAMES

In [507]:
df_clean.to_csv('../DATA/OUT/dataset_hors_outliers_surfaces.csv', index=False)

# DEFINITION DES OUTLIERS POUR LE NOMBRE DE PIECES PRINCIPALES

# CALCULS DES QUARTILES POUR NOS OUTLIERS

In [508]:
# CALCUL DE NOTRE PREMIER QUARTILE
q1_nb_piece = df['NombrePiecesPrincipales'].quantile([.25])
q1_nb_piece = q1_nb_piece[0.25]
q3_nb_piece = df['NombrePiecesPrincipales'].quantile([.75])
q3_nb_piece = q3_nb_piece[0.75]

# CALCUL DE NOTRE ECART INTER QUARTILES
iqr_nb_piece = q3_nb_piece - q1_nb_piece

print(f"q1 = {q1_nb_piece}, q3 = {q3_nb_piece}, IQR = {iqr_nb_piece}")

q1 = 2.0, q3 = 4.0, IQR = 2.0


# CALCUL DES VALEURS LIMITE OUTLIERS

In [509]:
outliers_nb_pieces_inferieur = q1_nb_piece - (1.5 * iqr_nb_piece)
outliers_nb_pieces_superieur = q3_nb_piece + (1.5 * iqr_nb_piece)

print(f'Outliers Inférieur = {outliers_nb_pieces_inferieur}')
print(f'Outliers Supérieur = {outliers_nb_pieces_superieur}')

Outliers Inférieur = -1.0
Outliers Supérieur = 7.0


# COMPATAGE DES OUTLIERS

In [510]:
nb_outliers_nb_piece = df.loc[(df['NombrePiecesPrincipales'] > outliers_nb_pieces_superieur) | (df['NombrePiecesPrincipales'] < outliers_nb_pieces_inferieur)].shape
print(f'Nous aurions donc {nb_outliers_nb_piece[0]} outliers.')

Nous aurions donc 2259 outliers.


# STOCKAGE DES OUTLIERS DANS UN NOUVEAU DATAFRAME

In [511]:
outliers_nb_pieces = df.loc[(df['NombrePiecesPrincipales'] > outliers_nb_pieces_superieur) | (df['NombrePiecesPrincipales'] < outliers_nb_pieces_inferieur)]

# CREATION D'UN DATAFRAME HORS OUTLIERS ET DOUBLONS D'OUTLIERS

In [512]:
# Concaténation des ID des outliers
ids_outliers = pd.concat([outliers_surface_bati['ID'], outliers_surface_terrain['ID'], outliers_surface_carrez['ID'], outliers_nb_pieces['ID']])

# Suppression des duplicatas
ids_outliers = ids_outliers.drop_duplicates()

# Filtrage de df pour exclure les outliers
df_clean_with_nb_piece = df[~df['ID'].isin(ids_outliers)]

# CREATION D'UN CSV DEPUIS CE DATAFRAMES

In [513]:
df_clean_with_nb_piece.to_csv('../DATA/OUT/dataset_hors_outliers_surfaces_and_piece.csv', index=False)